# Analyzing Ptolemy's geographic data

## Author: Kristen Qako

## Overview

This notebook will help you adjust Ptolemy's values for longitude and latitude to account for:

- his mistakenly small dimension of the earth's circumference
- his origin of longitude (ca. 12.8 degrees west of Greenwich)
- his use of the "parallel through Rhodes" (36 degrees north latitude in Ptolemy's data) as the baseline for computing latitude values.


## Using your adjusted data in a GIS

You could use the contents of this notebook in several ways:

1. run as a Jupyter notebook directly (either on mybinder.org, or downloaded and run with software like nteract)
2. download the notebook as Scala, open the Scala content in Atom, and run the code directly there.

In either case, you will want to write your adjusted data to a `.csv` file you can use directly in a GIS.

A Jupyter notebook on mybinder won't have access to your host computer's file system, so you'll have to print out the values in your notebook, and copy and paste them in to a file on your computer.  If you're running the code in a local environment like Atom, you can write the output directly from your Scala code.  The instructions at the end of this notebook will show you how to do both of these things.


## Load data for Ptolemy

You can use an existing code library to read an XML edition of Ptolemy's *Geography* and extract the 6,000 geographic points into a class of object that will make them straightforward to work with.

In [1]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)


myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [2]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::ptolemy:1.2.1`
import scala.xml._

Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/ptolemy_2.12/1.2.1/ptolemy_2.12-1.2.1.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/ptolemy_2.12/1.2.1/ptolemy_2.12-1.2.1.pom.sha1
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/ptolemy_2.12/1.2.1/ptolemy_2.12-1.2.1.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cite/xcite_2.12/4.0.2/xcite_2.12-4.0.2.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cite/xcite_2.12/4.0.2/xcite_2.12-4.0.2.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/histoutils_2.12/2.2.0/histoutils_2.12-2.2.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/greek_2.12/4.0.1/greek_2.12-4.0.1.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.4/scala-library-2.12.4.pom
Downloaded https://repo1.maven.org/maven2/org/wvlet/airframe/airframe-log_2.12/19.9.0/airframe-log_2.12-19.9.0.pom
Downloaded https://dl.bintray.com/neels

Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cite/xcite_2.12/4.2.0/xcite_2.12-4.2.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/dse_2.12/5.3.0/dse_2.12-5.3.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/citebinaryimage_2.12/3.1.1/citebinaryimage_2.12-3.1.1.pom.sha1
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/cex_2.12/6.3.3/cex_2.12-6.3.3.pom
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/xmlutils_2.12/2.0.0/xmlutils_2.12-2.0.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/citeobj_2.12/7.4.0/citeobj_2.12-7.4.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/scm_2.12/7.0.1/scm_2.12-7.0.1.pom.sha1
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/dse_2.12/5.3.0/dse_2.12-5.3.0.pom
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/cite/xcite_2.12/4.2.0/xcite_2.12-4.2.0.pom
Downloaded https://dl.bintra

Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.0.6/scala-xml_2.12-1.0.6-sources.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/midvalidator_2.12/10.0.0/midvalidator_2.12-10.0.0.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/xmlutils_2.12/2.0.0/xmlutils_2.12-2.0.0.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/cite/xcite_2.12/4.2.0/xcite_2.12-4.2.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.1.1/scala-collection-compat_2.12-2.1.1-sources.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/cex_2.12/6.4.0/cex_2.12-6.4.0-sources.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/seqcomp_2.12/1.0.0/seqcomp_2.12-1.0.0-sources.jar
Downloaded https://repo1.maven.org/maven2/ch/qos/logback/logback-core/1.2.3/logback-core-1.2.3-sources.jar
Downloaded https://repo1.maven.org/maven2

import $ivy.$                                  

import scala.xml._

In [3]:
// read and parse XML file of Ptolemy:
val url = "https://raw.githubusercontent.com/neelsmith/ptolemy/master/tei/tlg0363.tlg009.epist03-p5-u8.xml"
val root = XML.load(url)

url: String = "https://raw.githubusercontent.com/neelsmith/ptolemy/master/tei/tlg0363.tlg009.epist03-p5-u8.xml"
root: Elem = <TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>Ptolemy, Geography</title>
            </titleStmt>
            <publicationStmt>
                <p>E-text</p>
            </publicationStmt>
            <sourceDesc>
                <p>Composite based on print editions of Müller and Nobbe.</p>
            </sourceDesc>
        </fileDesc>
        <profileDesc>
            <langUsage>
                <language ident="grc">Greek</language>
                <language ident="eng">English</language>
            </langUsage>
        </profileDesc>
    </teiHeader>
    <text>
        <body xml:lang="grc">


            <interpGrp type="continents">
                <interp xml:id="Europe"/>
                <interp xml:id="Asia"/>
                <interp xml:id="Libya"/>
            </interpGrp>

  

In [4]:
// parse XML text into objects
import edu.holycross.shot.ptolemy._
val delimited = TeiParser.parseTEI(root, false)
val ptolemyPoints = delimited.map(ln => PtolemyString(ln))

2020-03-31 18:12:41.070Z error [TeiParser] No name element in   - (TeiParser.scala:98)
2020-03-31 18:12:41.118Z error [TeiParser] No name element in   - (TeiParser.scala:98)
2020-03-31 18:12:41.120Z error [TeiParser] No name element in   - (TeiParser.scala:98)
2020-03-31 18:12:41.120Z error [TeiParser] No name element in   - (TeiParser.scala:98)


import edu.holycross.shot.ptolemy._

delimited: Vector[String] = Vector(
  "2.2.1#Europe#hibernia#paralios#pt_ll_1#\u0392\u1f79\u03c1\u03b5\u03b9\u03bf\u03bd \u1f04\u03ba\u03c1\u03bf\u03bd#\u03b9\u03b1\u02b9#\u03be\u03b1\u02b9#11.0#11#0.0#61.0#61#0.0",
  "2.2.1#Europe#hibernia#paralios#pt_ll_2#\u039f\u1f50\u03b5\u03bd\u03bd\u1f77\u03ba\u03bd\u03b9\u03bf\u03bd \u1f04\u03ba\u03c1\u03bf\u03bd#\u03b9\u03b2\u02b9\u03b2  \u03b3\"#\u03be\u03b1\u02b9\u03b3\"#12.833#12#0.833#61.333#61#0.333",
  "2.2.1#Europe#hibernia#paralios#pt_ll_3#\u039f\u1f50\u03b9\u03b4\u03bf\u1f7b\u03b1 \u03c0\u03bf\u03c4\u03b1\u03bc\u03bf\u1fe6 \u1f10\u03ba\u03b2\u03bf\u03bb\u03b1\u1f77#\u03b9\u03b3\u02b9#\u03be\u03b1\u02b9#13.0#13#0.0#61.0#61#0.0",
  "2.2.1#Europe#hibernia#paralios#pt_ll_4#\u1f08\u03c1\u03b3\u1f77\u03c4\u03b1 \u03c0\u03bf\u03c4\u03b1\u03bc\u03bf\u1fe6 \u1f10\u03ba\u03b2\u03bf\u03bb\u03b1\u1f77#\u03b9\u03b4\u02b9\u03b2\"#\u03be\u03b1\u02b9\u03b2\"#14.5#14#0.5#61.5#61#0.5",
  "2.2.1#Europe#hibernia#parali

Each of the `ptolemyPoints` objects has a `lon` and a `lat` member.

Look at the example of a single point in following cell to figure out what class the `lon` and `lat` members belong to.

In [5]:
val firstPoint = ptolemyPoints(0)
firstPoint.id
firstPoint.lon
firstPoint.lat
firstPoint.continent
firstPoint.province

firstPoint: PtolemyString = PtolemyString(
  "2.2.1",
  "Europe",
  "hibernia",
  "paralios",
  "pt_ll_1",
  "\u0392\u1f79\u03c1\u03b5\u03b9\u03bf\u03bd \u1f04\u03ba\u03c1\u03bf\u03bd",
  "\u03b9\u03b1\u02b9",
  "\u03be\u03b1\u02b9",
  11.0,
  "11",
  "0.0",
  11.0,
  "61",
  "0.0"
)
res4_1: String = "pt_ll_1"
res4_2: Double = 11.0
res4_3: Double = 11.0
res4_4: String = "Europe"
res4_5: String = "hibernia"

## 1. Scale the data

As you know from your background reading, we will use the ratio of Eratosthenes' figure for the circumference of the earth to Ptolemy's figure to scale the longitude and latitude values down by about 72%.

In [6]:
val ratio = 18.0 / 25.0
// We'll take one arbitrary point as an example
// Here's an example:
firstPoint.lon
firstPoint.lon * ratio
firstPoint.lat
firstPoint.lat * ratio


ratio: Double = 0.72
res5_1: Double = 11.0
res5_2: Double = 7.92
res5_3: Double = 11.0
res5_4: Double = 7.92



To simplify your work, you could work just with the longitude and latitude values for each point.  Scala's case class is a natural way to accomplish that.

The following cell defines a class named `GeoPoint` that has three members, plus one function to format the contents as a comma-separated String. It shows how you can create instances of that class.

In [7]:
case class GeoPoint (id: String, lon: Double, lat: Double) {
    def csv : String = {
        id + "," + lon + "," + lat
    }
}


val firstGeo = GeoPoint(firstPoint.id, firstPoint.lon, firstPoint.lat)
firstGeo.csv

defined class GeoPoint
firstGeo: GeoPoint = GeoPoint("pt_ll_1", 11.0, 11.0)
res6_2: String = "pt_ll_1,11.0,11.0"

This makes it very straightforward to map the Vector of ptolemy points to a new Vector of `GeoPoint` objects.

In [ ]:
val ptolemyGeo = ptolemyPoints.map(pt => GeoPoint(pt.id, pt.lon, pt.lat))

If we wanted to create a rescaled version of the first longitude, latitude pair, we could easily do that: 

In [ ]:
ptolemyGeo.size
ptolemyPoints.size

In [ ]:
val firstRescaled = GeoPoint(firstPoint.id, firstPoint.lon * ratio, firstPoint.lat * ratio)

### Task: create a Vector of rescaled points

Now create a Vector of `GeoPoint` objects.  Verify that you have the same number of them as the size of your original Vector of Ptolemy points.

In [ ]:
val ptolemyRescaled = ptolemyGeo.map( pt => GeoPoint(pt.id, pt.lon * ration, pt.lat * ratio))

## 2. Adjust origin of longitude

Empirical comparison suggests that Ptolemy's origin of longitude was about 12.8 degrees west of Greenwich.

The following cell creates a `GeoPoint` adjusting Ptolemy's origin of longitude to align with our origin of longitude.


In [ ]:
// negative because Ptolemy's 0 is *west* of Greenwich:
val originLongitude = -12.8
firstRescaled
val firstLonAdjusted = GeoPoint(firstRescaled.id, firstRescaled.lat, firstRescaled.lon + originLongitude)

### Task: create a Vector of points with adjusted longitude


In [ ]:
// Map your existing ptolemyRescaled Vector:

val ptolemyLonAdjusted = ptolemyScaled.map( pt => ??? )

## 3. Adjust base of latitude

When Ptolemy converted ground distances to spherical coordinates, he did not use the equator (0 degrees of latitude) as his baseline to compute from.  Instead, he used "the parallel through Rhodes," which he gives as 36 degrees north of the equator.  But if we scale his raw value of 36 degrees by the ratio of 18/25, then the baseline he thought was 36 degrees north of the equator was actually less than 26 degrees north of the equator.  We need to *add* to each latitude value this difference (roughly 10 degrees) between the raw value of 36 degrees and the scaled-down value.

The following cell shows how to compute that offset value, and apply it to one point.

In [ ]:
val rhodesRaw = 36.0
val rhodesAdjusted = ratio * rhodesRaw
val offset = rhodesRaw - rhodesAdjusted


In [ ]:
val firstLonLatAdjusted = GeoPoint(firstLonAdjusted.id, firstLonAdjusted.lat + offset, firstLonAdjusted.lon)

### Task: create a Vector of points with all three adjustments

In [ ]:
// Map the existing ptolemyLonAdjusted Vector:

val ptolemyAdjusted = 
    ptolemyLonAdjusted.map(pt => GeoPoint(pt.id, 
                                          pt.lat + offset, 
                                          pt.lon))

ptolemyAdjusted.size



## Get your data into a GIS

We'd like to write a file with our data in `.csv` format that a GIS can read.

This requires two steps:

1. format the Vector of `GeoPoint` objects as csv Strings.
2. write the formatted Strings to a file

The `csv` method of the `GeoPoint` class will simplify this: we can simply map every `GeoPoint` to the String output of its `csv` method.

In [ ]:
val csvVector = ptolemyAdjusted.map(pt => pt.csv)

Vectors have a handy `mkString` method to make a String out of a Vector.  It takes one parameter:  a String value used to join each element.  The following cell turns the Vector of Strings into a single String with new lines separating the components of the source Vector.

In [ ]:
val csv = csvVector.mkString("\n")

We should define a header line to include in our `csv` file:

In [ ]:
val header = "id,lon,lat\n"

### If running locally (e.g., in Atom)

`PrintWriter` is a clunky old Java class but if you just clone the code in the following cell, it's easy enough to write your output to a file in your local file system.

In [ ]:
import java.io.PrintWriter
new PrintWriter("ptolemy-output.csv"){ write(header + csv); close; }

### If running Jupyter notebook on `mybinder.org`

If you're running the Jupyter notebook on `mybinder.org`, use `println` to display all values, that you can then (tediously) copy and paste into a text file.

In [ ]:
println(header + csv)

## Load your CSV file into QGIS and visualize!
